<a href="https://colab.research.google.com/github/ichekhovskikh/doc2vec/blob/master/RecommendationSystem.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install pymorphy2[fast]

import gensim
import os
import collections
import smart_open
import random
import json
import urllib.request
import pymorphy2
import nltk

nltk.download('stopwords')

from nltk.corpus import stopwords
from pymystem3 import Mystem
from IPython.display import HTML, display
from tabulate import tabulate

## Опредлим функцию для чтения и предварительной обработки текста
Ниже мы определяем функцию для открытия  train/test файла, предварительно обрабатываем каждый текст датасета, используя простой инструмент предварительной обработки gensim (то есть, разбиваем текст на отдельные слова, удалите знаки препинания, установите строчные буквы и т. д.), лемматизацию, удаление стоп слов и возвращаем список слов. Для обучения модели нам нужно будет связать тег с каждым документом учебного корпуса. В нашем случае тег - это идентификатор статьи.

Лемматизация каждого слова статьи:

In [0]:
morph = pymorphy2.MorphAnalyzer()

def lemmatize(words):
    for word in words:
        yield morph.parse(word)[0].normal_form

Удаление стоп слов:

In [0]:
russian_stopwords = stopwords.words("russian")

def remove_stopwords(words):
    return [word for word in words if word not in russian_stopwords]

Предобработка текста статьи:

In [0]:
def advanced_preprocess(text):
    normalized_text = gensim.utils.simple_preprocess(text)
    normalized_text = list(lemmatize(normalized_text))
    normalized_text = remove_stopwords(normalized_text)
    return normalized_text

Отрытие файла с корпусом статей:

In [0]:
def read_corpus(corpus_path, tokens_only=False):
    with urllib.request.urlopen(corpus_path) as corpus_url:
        corpus = json.loads(corpus_url.read().decode())
        for article in corpus:
            normalized_text = advanced_preprocess(article['text'])
            if tokens_only:
                yield normalized_text
            else:
                yield gensim.models.doc2vec.TaggedDocument(normalized_text, article['tags'])

Получение исходного текста статьи по индексу:

In [0]:
def get_article_text_by_index(index, corpus_path):
    with urllib.request.urlopen(corpus_path) as corpus_url:
        corpus = json.loads(corpus_url.read().decode())
        return corpus[index]['text']

Получение исходного текста статьи по идентификатору:

In [0]:
def get_article_text_by_id(id, corpus_path):
    with urllib.request.urlopen(corpus_path) as corpus_url:
        corpus = json.loads(corpus_url.read().decode())
        for index in range(len(corpus)):
          if (corpus[index]['id'] == id):
            return corpus[index]['text']
        return ""

Получение категории статьи по индексу:

In [0]:
def get_article_class_by_index(index, corpus_path):
    with urllib.request.urlopen(corpus_path) as corpus_url:
        corpus = json.loads(corpus_url.read().decode())
        return corpus[index]['class_name']

Получение категории статьи по идентификатору:

In [0]:
def get_article_class_by_id(id, corpus_path):
    with urllib.request.urlopen(corpus_path) as corpus_url:
        corpus = json.loads(corpus_url.read().decode())
        for index in range(len(corpus)):
          if (corpus[index]['id'] == id):
            return corpus[index]['class_name']
        return ""

#Начинаем
Для начала нам понадобится комплект документов для обучения нашей модели doc2vec. Теоретически, документ может быть чем угодно: коротким твитом из 140 символов, отдельным абзацем, новостной статьей или книгой. В NLP комплект документов часто называют корпусом.

Будем тренировать нашу модель на собственном корпусе. Этот корпус содержит 500 научных статей на 5 различных тем.

Для тестирования будет использоваться тестовый корпус из 50 статей (10 статей на каждую тему).

Dataset состоит из трех строк: id (идентификатор строки), text (текст статьи), tag (идентификатор самой статьи, вектор которого будем обучать), class_name (класс текста, был размечен вручную, необходим только для тестирования)

In [0]:
#@title Введите путь к файлам исходной базы статей:
train_path = 'https://raw.githubusercontent.com/ichekhovskikh/cyberleninka-article-downloader/master/train_corpus.json' #@param {type: "string"}
test_path = 'https://raw.githubusercontent.com/ichekhovskikh/cyberleninka-article-downloader/master/test_corpus.json' #@param {type: "string"}

train_corpus = []
test_corpus = []

try:
    train_corpus = list(read_corpus(train_path))
    test_corpus = list(read_corpus(test_path, tokens_only=True))
    print("Загружено.")
except:
    print("Невозможно открыть файл! Проверьте формат выбранных файлов.")

Давайте посмотрим на учебный корпус:

In [0]:
print(train_corpus[:2])

Корпус тестирования выглядит так:

In [0]:
print(test_corpus[:2])

Обратите внимание, что корпус тестирования представляет собой просто список списков и не содержит никаких тегов.

# Обучение модели
## Создание объекта Doc2Vec
Теперь мы создадим модель Doc2Vec с векторным размером 80 слов и перебираем учебный корпус 200 раз (данные параметры были получены после проведения ряда исследований).

Словарь содержит в себе все уникальные слова, извлеченных из учебного корпуса.

In [0]:
#@title Укажите параметры обучения модели:
vector_size = 80 #@param
window =  3 #@param
epochs =  200 #@param
alpha = 0.001 #@param
learning_method = 'PV-DM' #@param ["PV-DM", "PV-DBOW"] {type:"string"}

dm = 1
if (learning_method == 'PV-DBOW'):
    dm = 0

model = gensim.models.doc2vec.Doc2Vec(
    vector_size=vector_size, 
    dm=dm,
    window=window,
    alpha=alpha, 
    epochs=epochs)

model.build_vocab(train_corpus)
%time model.train(train_corpus, total_examples=model.corpus_count, epochs=model.epochs)

## Inferring a Vector
Важно отметить, что теперь вы можете вывести вектор для любого фрагмента текста без необходимости переобучать модель, передав список слов в функцию model.infer_vector. Затем этот вектор можно сравнить с другими векторами по косинусной близости.

In [0]:
text = 'Горностай небольшой зверек семейства куньих очень ценится'
model.infer_vector(advanced_preprocess(text))

Обратите внимание, что, поскольку лежащие в основе алгоритмы обучения представляют собой подходы на основе итеративной аппроксимации, в которой используется внутренняя рандомизация, таким обрахрм, повторные выводы одного и того же текста будут возвращать слегка разные векторы.

# Оценочная модель
Чтобы оценить нашу новую модель, мы сначала выведем новые векторы для каждого документа тренировочного корпуса, сравним выведенные векторы с тренировочным корпусом.

Проверка выведенного вектора по обучающему вектору является своего рода «проверкой работоспособности» в отношении того, ведет ли модель себя адекватно, хотя и не является реальным значением «точности».

Можем взглянуть на пример:

In [0]:
doc_id = random.randint(0, len(train_corpus) - 1)

inferred_vector = model.infer_vector(train_corpus[doc_id].words)
sims = model.docvecs.most_similar([inferred_vector], topn=5)
tabledata = [["Индекс", "Схожесть", "Категория", "Текст"]]

print('\nТЕКСТ ИСХДНОГО ДОКУМЕНТА ({}) «{}»: «{}»\n'.format(doc_id, get_article_class_by_id(doc_id, train_path), get_article_text_by_index(doc_id, train_path)))
for label, index in [('1)', 0), ('2)', 1), ('3)', 2), ('4)', 3), ('5)', 4)]:
    tabledata.append([label, sims[index], get_article_class_by_id(int(sims[index][0]), train_path), get_article_text_by_id(int(sims[index][0]), train_path)])

print(tabulate(tabledata))

Обратите внимание, что наиболее похожий документ (как правило, тот же текст) имеет параметр сходства, приближающийся к единице. 

# Тестирование модели
Используя тот же подход, что и выше, мы выведем вектор для случайно выбранного тестового документа и сравним документ с нашей моделью с помощью f1-меры.

In [0]:
#@title Выполнить тестирование обученной модели

from sklearn.metrics import f1_score

test_size = len(test_corpus)

predicted_classes = [] 
test_classes = []

for doc_index in range(test_size):
    inferred_vector = model.infer_vector(test_corpus[doc_index])
    sims = model.docvecs.most_similar([inferred_vector], topn=5)

    test_article_class = get_article_class_by_index(doc_index, test_path)
    tabledata = [["Индекс", "Схожесть", "Категория", "Текст"]]
    print('\nТЕКСТ ИСХДНОГО ДОКУМЕНТА ({}) «{}»: «{}»\n'.format(doc_index, test_article_class, get_article_text_by_index(doc_index, test_path)))
    for label, index in [('1)', 0), ('2)', 1), ('3)', 2), ('4)', 3), ('5)', 4)]:
        predicted_article_class = get_article_class_by_id(int(sims[index][0]), train_path)
        tabledata.append([label, sims[index], predicted_article_class, get_article_text_by_id(int(sims[index][0]), train_path)])
        predicted_classes.append(predicted_article_class)
        test_classes.append(test_article_class)
    print(tabulate(tabledata))

print('f1score: {}'.format(f1_score(test_classes, predicted_classes, average='macro')))

# Поиск похожих научных документов
Выполните поиск похожих научных статей

In [0]:
#@title Укажите путь к тексту статьи в формате *.txt или введите текст статьи:
article_text = 'TECHNICAL SCIENCE \u0423\u0414\u041A 004.912 \u041C\u0430\u043B\u044C\u043A\u043E\u0432 \u0410. \u0410. \u0422\u0432\u0435\u0440\u0441\u043A\u043E\u0439 \u0433\u043E\u0441\u0443\u0434\u0430\u0440\u0441\u0442\u0432\u0435\u043D\u043D\u044B\u0439 \u0442\u0435\u0445\u043D\u0438\u0447\u0435\u0441\u043A\u0438\u0439 \u0443\u043D\u0438\u0432\u0435\u0440\u0441\u0438\u0442\u0435\u0442 DOI: 10.24411/2520-6990-2019-10113 \u041C\u0415\u0422\u041E\u0414\u042B \u041F\u041E\u0421\u0422\u0420\u041E\u0415\u041D\u0418\u042F \u041A\u041B\u0410\u0421\u0421\u0418\u0424\u0418\u041A\u0410\u0422\u041E\u0420\u0410 \u0422\u0415\u0425\u041D\u0418\u0427\u0415\u0421\u041A\u041E\u0419 \u0414\u041E\u041A\u0423\u041C\u0415\u041D\u0422\u0410\u0426\u0418\u0418 Malkov A. A. Tver state technical University METHODS FOR CONSTRUCTING CLASSIFIER TECHNICAL DOCUMENTATIONS \u0410\u043D\u043D\u043E\u0442\u0430\u0446\u0438\u044F. \u041F\u0440\u0438\u0432\u043E\u0434\u0438\u0442\u0441\u044F \u043F\u043E\u0441\u0442\u0430\u043D\u043E\u0432\u043A\u0430 \u0437\u0430\u0434\u0430\u0447\u0438 \u043A\u043B\u0430\u0441\u0441\u0438\u0444\u0438\u043A\u0430\u0446\u0438\u0438 \u043F\u0440\u0438\u043C\u0435\u043D\u0438\u0442\u0435\u043B\u044C\u043D\u043E \u0434\u043B\u044F \u043F\u043E\u0441\u0442\u0440\u043E\u0435\u043D\u0438\u044F \u0441\u0438\u0441\u0442\u0435\u043C\u044B \u0430\u0432\u0442\u043E\u043C\u0430\u0442\u0438\u0447\u0435\u0441\u043A\u043E\u0439 \u043A\u043B\u0430\u0441\u0441\u0438\u0444\u0438\u043A\u0430\u0446\u0438\u0438 \u0442\u0435\u043A\u0441\u0442\u043E\u0432\u044B\u0445 \u0434\u043E\u043A\u0443\u043C\u0435\u043D\u0442\u043E\u0432. \u0420\u0430\u0441\u0441\u043C\u0430\u0442\u0440\u0438\u0432\u0430\u044E\u0442\u0441\u044F \u043C\u0435\u0442\u043E\u0434\u044B \u0440\u0435\u0448\u0435\u043D\u0438\u044F \u0437\u0430\u0434\u0430\u0447\u0438 \u0430\u0432\u0442\u043E\u043C\u0430\u0442\u0438\u0447\u0435\u0441\u043A\u043E\u0439 \u043A\u043B\u0430\u0441\u0441\u0438\u0444\u0438\u043A\u0430\u0446\u0438\u0438 \u0442\u0435\u043A\u0441\u0442\u043E\u0432\u044B\u0445 \u0434\u043E\u043A\u0443\u043C\u0435\u043D\u0442\u043E\u0432. \u041F\u0440\u0438\u0432\u043E\u0434\u0438\u0442\u0441\u044F \u043F\u043E\u0434\u0445\u043E\u0434 \u043A \u043F\u043E\u0441\u0442\u0440\u043E\u0435\u043D\u0438\u044E \u0441\u0438\u0441\u0442\u0435\u043C\u044B \u0430\u0432\u0442\u043E\u043C\u0430\u0442\u0438\u0447\u0435\u0441\u043A\u043E\u0439 \u043A\u043B\u0430\u0441\u0441\u0438\u0444\u0438\u043A\u0430\u0446\u0438\u0438. Abstract. The formulation of the classification problem for the construction of the system of automatic classification of text documents is given. Methods of solving the problem of automatic classification of text documents are considered. An approach to the construction of an automatic classification system is given . \u0442\u043E\u0432. \u041A\u043B\u044E\u0447\u0435\u0432\u044B\u0435 \u0441\u043B\u043E\u0432\u0430: \u043A\u043B\u0430\u0441\u0441\u0438\u0444\u0438\u043A\u0430\u0446\u0438\u044F, \u0442\u0435\u043A\u0441\u0442\u043E\u0432\u044B\u0439 \u0434\u043E\u043A\u0443\u043C\u0435\u043D\u0442, \u043C\u0430\u0448\u0438\u043D\u043D\u043E\u0435 \u043E\u0431\u0443\u0447\u0435\u043D\u0438\u0435, \u0438\u043D\u0434\u0435\u043A\u0441\u0430\u0446\u0438\u044F \u0434\u043E\u043A\u0443\u043C\u0435\u043D-Key words: classification, text document, machine learning, document indexing. \u041F\u043E\u0441\u0442\u0430\u043D\u043E\u0432\u043A\u0430 \u0437\u0430\u0434\u0430\u0447\u0438 \u043A\u043B\u0430\u0441\u0441\u0438\u0444\u0438\u043A\u0430\u0446\u0438\u0438 \u043F\u043E \u0437\u0430\u0434\u0430\u043D\u043D\u043E\u0439 \u043A\u0430\u0442\u0435\u0433\u043E\u0440\u0438\u0438 ct \u0434\u043B\u044F i=1,..., |C|. \u0412 \u0442\u0430\u043A\u043E\u043C \u041A\u043B\u0430\u0441\u0441\u0438\u0444\u0438\u043A\u0430\u0446\u0438\u044F \u0434\u043E\u043A\u0443\u043C\u0435\u043D\u0442\u043E\u0432 \u044F\u0432\u043B\u044F\u0435\u0442\u0441\u044F \u0437\u0430\u0434\u0430\u0447\u0435\u0439 \u043E\u043F\u0440\u0435\u0434\u0435\u043B\u0435\u043D\u0438\u044F \u043D\u0435\u043A\u043E\u0442\u043E\u0440\u043E\u0433\u043E \u0431\u0443\u043B\u0435\u0432\u0430 \u0437\u043D\u0430\u0447\u0435\u043D\u0438\u044F \u0434\u043B\u044F \u043A\u0430\u0436- \u0441\u043B\u0443\u0447\u0430\u0435 \u043A\u043B\u0430\u0441\u0441\u0438\u0444\u0438\u043A\u0430\u0442\u043E\u0440\u043E\u043C \u0434\u043B\u044F ct \u044F\u0432\u043B\u044F\u0435\u0442\u0441\u044F \u0444\u0443\u043D\u043A\u0446\u0438\u044F \u0434\u043E\u0438 \u043F\u0430\u0440\u044B \u0439, \u0441,) \u0435 \u0411 \u0445 \u0421 , \u0433\u0434\u0435 D - \u043C\u043D\u043E\u0436\u0435\u0441\u0442\u0432\u043E \u043A\u043B\u0430\u0441\u0441\u0438\u0444\u0438\u0446\u0438\u0440\u0443\u0435\u043C\u044B\u0445 \u0434\u043E\u043A\u0443\u043C\u0435\u043D\u0442\u043E\u0432, \u0430 \u0421 ~ {\u04411,---, \u0441|\u0441|} \u044F\u0432\u043B\u044F\u0435\u0442\u0441\u044F \u043C\u043D\u043E\u0436\u0435\u0441\u0442\u0432\u043E\u043C \u043F\u0440\u0435\u0434- \u0424 : \u0411 ^ {\u0422, \u0420}, \u0430\u043F\u043F\u0440\u043E\u043A\u0441\u0438\u043C\u0438\u0440\u0443\u044E\u0449\u0430\u044F \u043D\u0435\u0438\u0437\u0432\u0435\u0441\u0442\u043D\u0443\u044E \u0446\u0435\u043B\u0435\u0432\u0443\u044E \u0444\u0443\u043D\u043A\u0446\u0438\u044E \u0424. : \u0411 ^ {\u0422, \u0411} . \u0417\u043D\u0430\u0447\u0435\u043D\u0438\u0435 \u0422, \u043E\u0442\u043D\u043E\u0441\u0438\u0442\u0435\u043B\u044C\u043D\u043E (\u0439 \u25A0, \u0441\u0433) , \u043E\u0437\u043D\u0430\u0447\u0430\u0435\u0442 \u0440\u0435\u0448\u0435\u043D\u0438\u0435 \u043E \u043E\u043F\u0440\u0435\u0434\u0435\u043B\u0435\u043D\u043D\u044B\u0445 \u043A\u0430\u0442\u0435\u0433\u043E\u0440\u0438\u0438. \u0412 \u0441\u043B\u0443\u0447\u0430\u0435, \u043A\u043E\u0433\u0434\u0430 \u043A\u0430\u0436\u0434\u043E\u043C\u0443 \u0441\u043E\u043E\u0442\u0432\u0435\u0442\u0441\u0442\u0432\u0438\u0438 \u0434\u043E\u043A\u0443\u043C\u0435\u043D\u0442\u0430 \u043A\u0430\u0442\u0435\u0433\u043E\u0440\u0438\u0438 \u04411, \u0432 \u0442\u043E \u0434\u043E\u043A\u0443\u043C\u0435\u043D\u0442\u0443 \u0435 \u0411 \u0434\u043E\u043B\u0436\u043D\u0430 \u0431\u044B\u0442\u044C \u043F\u0440\u0438\u0441\u0432\u043E\u0435\u043D\u0430 \u043B\u0438\u0448\u044C \u0432\u0440\u0435\u043C\u044F \u043A\u0430\u043A, \u0437\u043D\u0430\u0447\u0435\u043D\u0438\u0435 F \u043E\u0437\u043D\u0430\u0447\u0430\u0435\u0442 \u0440\u0435\u0448\u0435\u043D\u0438\u0435 \u043E \u043D\u0435\u0441\u043E\u043E\u0442- \u043E\u0434\u043D\u0430 \u043A\u0430\u0442\u0435\u0433\u043E\u0440\u0438\u044F, \u043E\u0442\u043D\u043E\u0441\u044F\u0442 \u043A \u043E\u0434\u0438\u043D\u043E\u0447\u043D\u043E\u0439 \u043A\u043B\u0430\u0441\u0441\u0438\u0444\u0438\u043A\u0430- \u0432\u0435\u0442\u0441\u0442\u0432\u0438\u0438 \u0434\u043E\u043A\u0443\u043C\u0435\u043D\u0442\u0430 \u043A\u0430\u0442\u0435\u0433\u043E\u0440\u0438\u0438 \u0421\u0433. \u0446\u0438\u0438. \u0415\u0441\u043B\u0438 \u043B\u044E\u0431\u043E\u0435 \u043A\u043E\u043B\u0438\u0447\u0435\u0441\u0442\u0432\u043E \u043A\u0430\u0442\u0435\u0433\u043E\u0440\u0438\u0439 \u043C\u043E\u0436\u0435\u0442 \u043F\u0440\u0438- &#39; &#39; \u0441\u0432\u0430\u0438\u0432\u0430\u0442\u044C\u0441\u044F \u043A\u0430\u0436\u0434\u043E\u043C\u0443 \u0439. \u0435 \u0411, \u0442\u043E \u0433\u043E\u0432\u043E\u0440\u044F\u0442 \u043E \u043C\u043D\u043E\u0436\u0435\u0441\u0442\u0432\u0435\u043D\u043D\u043E\u0439 \u043A\u043B\u0430\u0441\u0441\u0438\u0444\u0438\u043A\u0430\u0446\u0438\u0438. \u041E\u0441\u043E\u0431\u044B\u043C \u0441\u043B\u0443\u0447\u0430\u0435\u043C \u043E\u0434\u0438\u043D\u043E\u0447\u043D\u043E\u0439 \u043A\u043B\u0430\u0441\u0441\u0438\u0444\u0438\u043A\u0430\u0446\u0438\u0438 \u044F\u0432\u043B\u044F\u0435\u0442\u0441\u044F \u0434\u0432\u043E\u0438\u0447\u043D\u0430\u044F \u0412 \u0442\u043E \u0432\u0440\u0435\u043C\u044F \u043A\u0430\u043A \u043F\u043E\u043B\u043D\u0430\u044F \u0430\u0432\u0442\u043E\u043C\u0430\u0442\u0438\u0437\u0430\u0446\u0438\u044F \u0437\u0430\u0434\u0430\u0447\u0438 \u043A\u043B\u0430\u0441\u0441\u0438\u0444\u0438\u043A\u0430\u0446\u0438\u0438 \u0442\u0440\u0435\u0431\u0443\u0435\u0442 \u043F\u0440\u0438\u043D\u044F\u0442\u0438\u044F \u0440\u0435\u0448\u0435\u043D\u0438\u044F \u043E\u0442\u043D\u043E\u0441\u0438- \u0442\u0435\u043B\u044C\u043D\u043E T \u0438 F \u0434\u043B\u044F \u043A\u0430\u0436\u0434\u043E\u0439 \u043F\u0430\u0440\u044B \u041Ed,, \u0421, ) \u0447\u0430\u0441\u0442\u0438\u0447\u043D\u0430\u044F \u043A\u043B\u0430\u0441\u0441\u0438\u0444\u0438\u043A\u0430\u0446\u0438\u044F, \u043F\u0440\u0438 \u043A\u043E\u0442\u043E\u0440\u043E\u0439 \u043A\u0430\u0436\u0434\u043E\u043C\u0443 \u044D\u043B\u0435\u043C\u0435\u043D\u0442\u0443 \u0430\u0432\u0442\u043E\u043C\u0430\u0442\u0438\u0437\u0430\u0446\u0438\u044F \u044D\u0442\u043E\u0433\u043E \u043F\u0440\u043E\u0446\u0435\u0441\u0441\u0430 \u043C\u043E\u0436\u0435\u0442 \u043D\u0430\u043A\u043B\u0430\u0434\u044B\u0432\u0430\u0442\u044C \u0439, \u0435 \u0411 \u0434\u043E\u043B\u0436\u043D\u0430 \u0431\u044B\u0442\u044C \u043F\u0440\u0438\u0441\u0432\u043E\u0435\u043D\u0430 \u043A\u0430\u0442\u0435\u0433\u043E\u0440\u0438\u044F \u04211, \u043B\u0438\u0431\u043E \u0440\u0430\u0437\u2122\u044B\u0435 \u0442\u0440\u0435\u0431\u043E\u0432\u0430\u043D\u0438\u044F. \u0421 \u043E\u0434\u043D\u043E\u0439 \u0441\u0442\u043E\u0440\u043E\u043D\u0443 \u0434\u043B\u044F \u0434\u0430\u043D\u043D\u043E\u0433\u043E \u0434\u043E\u043A\u0443\u043C\u0435\u043D\u0442\u0430 \u0439 \u0435 \u0411 \u0441\u0438\u0441\u0442\u0435\u043C\u0430 \u043C\u043E\u0436\u0435\u0442 \u043F\u0440\u043E\u0441\u0442\u043E \u0435\u0435 \u0434\u043E\u043F\u043E\u043B\u043D\u0435\u043D\u0438\u0435 \u0421. \u0414\u0432\u043E\u0438\u0447\u043D\u044B\u0439 \u0441\u043B\u0443\u0447\u0430\u0439 \u044F\u0432\u043B\u044F\u0435\u0442\u0441\u044F \u0431\u043E\u043B\u0435\u0435 \u043E\u0431\u0449\u0438\u043C, \u043D\u0435\u0436\u0435\u043B\u0438 \u043C\u043D\u043E\u0436\u0435\u0441\u0442\u0432\u0435\u043D\u043D\u044B\u0439, \u043F\u043E\u0441\u043A\u043E\u043B\u044C\u043A\u0443 \u043C\u043E\u0436\u043D\u043E \u043F\u0440\u0435\u043E\u0431\u0440\u0430\u0437\u043E\u0432\u0430\u0442\u044C \u0437\u0430\u0434\u0430\u0447\u0443 \u043C\u043D\u043E\u0436\u0435\u0441\u0442\u0432\u0435\u043D\u043D\u043E\u0439 \u043A\u043B\u0430\u0441\u0441\u0438\u0444\u0438\u043A\u0430\u0446\u0438\u0438 \u043F\u043E \u0440\u0430\u043D\u0436\u0438\u0440\u043E\u0432\u0430\u0442\u044C \u043A\u0430\u0442\u0435\u0433\u043E\u0440\u0438\u0438 \u043C\u043D\u043E\u0436\u0435\u0441\u0442\u0432\u0430 C = {\u04411,..., \u0421|\u0421|} \u0432 \u0441\u043E\u043E\u0442\u0432\u0435\u0442\u0441\u0442\u0432\u0438\u0438 \u0441 \u043E\u0446\u0435\u043D\u043A\u043E\u0439 \u043F\u0440\u0438\u043D\u0430\u0434\u043B\u0435\u0436\u043D\u043E\u0441\u0442\u0438 \u043A \u0434\u043E\u043A\u0443\u043C\u0435\u043D\u0442\u0443 d. \u0431\u0435\u0437 \u043F\u0440\u0438\u043D\u044F\u0442\u0438\u044F \u043A\u0430\u043A\u043E\u0433\u043E-\u043B\u0438\u0431\u043E \u0441\u0442\u0440\u043E\u0433\u043E\u0433\u043E \u0440\u0435\u0448\u0435\u043D\u0438\u044F. \u0421 \u0434\u0440\u0443\u0433\u043E\u0439 - \u0434\u043B\u044F \u0437\u0430\u0434\u0430\u043D-\u0434^ \u043D\u043E\u0439 \u043A\u0430\u0442\u0435\u0433\u043E\u0440\u0438\u0438 ct G C \u0441\u0438\u0441\u0442\u0435\u043C\u0430 \u043C\u043E\u0436\u0435\u0442 \u0440\u0430\u043D\u0436\u0438\u0440\u043E\u0432\u0430\u0442\u044C \u0434\u043E\u043A\u0443\u043C\u0435\u043D\u0442\u044B D \u0432 \u0441\u043E\u043E\u0442\u0432\u0435\u0442\u0441\u0442\u0432\u0438\u0438 \u0441 \u043E\u0446\u0435\u043D\u043A\u043E\u0439 \u043F\u0440\u0438\u043D\u0430\u0434\u043B\u0435\u0436\u043D\u043E\u0441\u0442\u0438 \u043A \u043A\u0430\u0442\u0435\u0433\u043E\u0440\u0438\u0438 c [1]. \u0417\u0430\u0434\u0430\u0447\u0430 \u043A\u043B\u0430\u0441\u0441\u0438\u0444\u0438\u043A\u0430\u0446\u0438\u0438 \u0434\u043E\u043A\u0443\u043C\u0435\u043D\u0442\u043E\u0432 \u043D\u0430\u0445\u043E\u0434\u0438\u0442\u0441\u044F ^ \u043D\u0430 \u043F\u0435\u0440\u0435\u0441\u0435\u0447\u0435\u043D\u0438\u0438 \u0434\u0438\u0441\u0446\u0438\u043F\u043B\u0438\u043D \u043C\u0430\u0448\u0438\u043D\u043D\u043E\u0433\u043E \u043E\u0431\u0443\u0447\u0435\u043D\u0438\u044F \u0438 |\u0421| \u0443\u0434\u0435\u043C \u0440\u0430\u0441\u0441\u043C\u0430\u0442\u0440\u0438\u0432\u0430\u0442\u044C \u043A\u0430\u043A 1 1 \u0438\u043D\u0444\u043E\u0440\u043C\u0430\u0446\u0438\u043E\u043D\u043D\u043E\u0433\u043E \u043F\u043E\u0438\u0441\u043A\u0430, \u044F\u0432\u043B\u044F\u0435\u0442\u0441\u044F \u0440\u043E\u0434\u0441\u0442\u0432\u0435\u043D\u043D\u043E\u0439 \u043D\u0435\u0437\u0430\u0432\u0438\u0441\u0438\u043C\u044B\u0445 \u0437\u0430\u0434\u0430\u0447 \u043A\u043B\u0430\u0441\u0441\u0438\u0444\u0438\u043A\u0430\u0446\u0438\u0438 \u0434\u043E\u043A\u0443\u043C\u0435\u043D\u0442\u043E\u0432 D \u0442\u0430\u043A\u0438\u043C \u0437\u0430\u0434\u0430\u0447\u0430\u043C \u043A\u0430\u043A, \u043D\u0430\u043F\u0440\u0438\u043C\u0435\u0440, \u0430\u043D\u0430\u043B\u0438\u0437 \u0442\u0435\u043A\u0441\u0442\u043E\u0432 (text {c1v-5 \u0421|\u0441|} \u0432 |\u0421| \u043D\u0435\u0437\u0430\u0432\u0438\u0441\u0438\u043C\u044B\u0445 \u0437\u0430\u0434\u0430\u0447 \u0434\u0432\u043E\u0438\u0447\u043D\u043E\u0439 \u043A\u043B\u0430\u0441\u0441\u0438\u0444\u0438\u043A\u0430\u0446\u0438\u0438 \u043F\u043E {\u0421\u0443, \u0421\u0443} \u0434\u043B\u044F i =1, ..., |C| \u044D\u0442\u043E\u0433\u043E \u0442\u0440\u0435\u0431\u0443\u0435\u0442\u0441\u044F, \u0447\u0442\u043E\u0431\u044B \u043A\u0430\u0442\u0435\u0433\u043E\u0440\u0438\u0438 \u0431\u044B\u043B\u0438 \u0441\u0442\u043E\u0445\u0430\u0441\u0442\u0438\u0447\u0435\u0441\u043A\u0438 \u043D\u0435\u0437\u0430\u0432\u0438\u0441\u0438\u043C\u044B \u0434\u0440\u0443\u0433 \u043E\u0442 \u0434\u0440\u0443\u0433\u0430 [1]. \u0412 \u0438\u0442\u043E\u0433\u0435 \u0437\u0430\u0434\u0430\u0447\u0443 \u043A\u043B\u0430\u0441\u0441\u0438\u0444\u0438\u043A\u0430\u0446\u0438\u0438 C = {ci,..., c|Ci} 30 TECHNICAL SCIENCE / mining) [1, 2], \u0430 \u043F\u043E\u0442\u043E\u043C\u0443 \u0441\u0443\u0449\u0435\u0441\u0442\u0432\u0443\u044E\u0442 \u0440\u0430\u0437\u043B\u0438\u0447\u043D\u044B\u0435 \u043F\u043E\u0434\u0445\u043E\u0434\u044B \u043A \u0437\u0430\u0434\u0430\u0447\u0435 \u043A\u043B\u0430\u0441\u0441\u0438\u0444\u0438\u043A\u0430\u0446\u0438\u0438 \u0434\u043E\u043A\u0443\u043C\u0435\u043D\u0442\u043E\u0432. \u041C\u0435\u0442\u043E\u0434\u044B \u043C\u0430\u0448\u0438\u043D\u043D\u043E\u0433\u043E \u043E\u0431\u0443\u0447\u0435\u043D\u0438\u044F \u041F\u043E\u0434\u0445\u043E\u0434 \u043C\u0430\u0448\u0438\u043D\u043D\u043E\u0433\u043E \u043E\u0431\u0443\u0447\u0435\u043D\u0438\u044F \u043E\u0441\u043D\u043E\u0432\u0430\u043D \u043D\u0430 \u043D\u0430\u043B\u0438\u0447\u0438\u0438 \u0438\u0441\u0445\u043E\u0434\u043D\u043E\u0433\u043E \u043D\u0430\u0431\u043E\u0440\u0430 \u0434\u043E\u043A\u0443\u043C\u0435\u043D\u0442\u043E\u0432 Q = {dx,..., d,Q,} \u0421 D \u043F\u043E C = {\u04411,..., \u0441,\u0441,}. \u043A\u043B\u0430\u0441\u0441\u0438\u0444\u0438\u0446\u0438\u0440\u043E\u0432\u0430\u043D\u043D\u043E\u0433\u043E 1,..., } . \u042D\u0442\u043E \u0437\u043D\u0430\u0447\u0438\u0442, \u0447\u0442\u043E \u0437\u043D\u0430\u0447\u0435\u043D\u0438\u044F \u043F\u043E\u043B\u043D\u043E\u0439 \u0444\u0443\u043D\u043A\u0446\u0438\u0438 \u0424: \u041E X \u0421 ^ {\u0422, \u041A} \u0438\u0437\u0432\u0435\u0441\u0442\u043D\u044B \u0434\u043B\u044F \u043A\u0430\u0436\u0434\u043E\u0439 \u043F\u0430\u0440\u044B (\u0439,, \u0441) \u0435\u041E\u0445 \u0421. \u0414\u043E\u043A\u0443\u043C\u0435\u043D\u0442 d. \u044F\u0432\u043B\u044F\u0435\u0442\u0441\u044F \u043F\u043E\u043B\u043E\u0436\u0438\u0442\u0435\u043B\u044C\u043D\u044B\u043C \u043F\u0440\u0438\u043C\u0435\u0440\u043E\u043C \u0421\u0433, \u0435\u0441\u043B\u0438 \u0435\u0441\u043B\u0438 \u044D\u0442\u043E \u0443\u0441\u043B\u043E\u0432\u0438\u0435 \u043D\u0435 \u0432\u044B\u043F\u043E\u043B\u043D\u044F\u0435\u0442\u0441\u044F, \u0442\u043E \u043F\u043E\u043B\u0443\u0447\u0435\u043D\u043D\u044B\u0435 \u0440\u0435\u0437\u0443\u043B\u044C\u0442\u0430\u0442\u044B \u0431\u0443\u0434\u0443\u0442 \u043D\u0435\u043F\u0440\u0430\u0432\u0434\u043E\u043F\u043E\u0434\u043E\u0431\u043D\u043E \u0445\u043E\u0440\u043E\u0448\u0438\u043C\u0438 \u0438 \u043D\u0435 \u043D\u043E\u0441\u044F\u0442 \u043D\u0430\u0443\u0447\u043D\u043E\u0433\u043E \u0445\u0430\u0440\u0430\u043A\u0442\u0435\u0440\u0430. \u0412 \u0443\u0441\u043B\u043E\u0432\u0438\u044F\u0445 \u043F\u0440\u0430\u043A\u0442\u0438\u0447\u0435\u0441\u043A\u043E\u0433\u043E \u0438\u0441\u043F\u043E\u043B\u044C\u0437\u043E\u0432\u0430\u043D\u0438\u044F, \u043F\u043E\u0441\u043B\u0435 \u0432\u044B\u0447\u0438\u0441\u043B\u0435\u043D\u0438\u044F \u044D\u0444\u0444\u0435\u043A\u0442\u0438\u0432\u043D\u043E\u0441\u0442\u0438, \u043A\u043B\u0430\u0441\u0441\u0438\u0444\u0438\u043A\u0430\u0442\u043E\u0440, \u043A\u0430\u043A \u043F\u0440\u0430\u0432\u0438\u043B\u043E, \u043F\u0435\u0440\u0435\u043E\u0431\u0443\u0447\u0430\u044E\u0442 \u043D\u0430 \u043F\u043E\u043B\u043D\u043E\u043C \u043D\u0430\u0431\u043E\u0440\u0435 \u043F\u043E\u0434\u0433\u043E\u0442\u043E\u0432\u043B\u0435\u043D\u043D\u044B\u0445 \u0434\u043E\u043A\u0443\u043C\u0435\u043D\u0442\u043E\u0432, \u0441 \u0446\u0435\u043B\u044C\u044E \u0443\u0432\u0435\u043B\u0438\u0447\u0435\u043D\u0438\u044F \u044D\u0444\u0444\u0435\u043A\u0442\u0438\u0432\u043D\u043E\u0441\u0442\u0438. \u0412 \u044D\u0442\u043E\u043C \u0441\u043B\u0443\u0447\u0430\u0435 \u0440\u0435\u0437\u0443\u043B\u044C\u0442\u0430\u0442\u044B \u043F\u0440\u0435\u0434\u044B\u0434\u0443\u0449\u0435\u0433\u043E \u0432\u044B\u0447\u0438\u0441\u043B\u0435\u043D\u0438\u044F \u044D\u0444\u0444\u0435\u043A\u0442\u0438\u0432\u043D\u043E\u0441\u0442\u0438 \u0441\u0442\u0430\u043D\u043E\u0432\u044F\u0442\u0441\u044F \u043F\u0435\u0441\u0441\u0438\u043C\u0438\u0441\u0442\u0438\u0447\u0435\u0441\u043A\u043E\u0439 \u043E\u0446\u0435\u043D\u043A\u043E\u0439 \u0440\u0435\u0430\u043B\u044C\u043D\u043E\u0439 \u043F\u0440\u043E\u0438\u0437\u0432\u043E\u0434\u0438\u0442\u0435\u043B\u044C\u043D\u043E\u0441\u0442\u0438, \u043F\u043E\u0441\u043A\u043E\u043B\u044C\u043A\u0443 \u0432 \u0438\u0442\u043E\u0433\u0435 \u043A\u043B\u0430\u0441\u0441\u0438\u0444\u0438\u043A\u0430\u0442\u043E\u0440 \u0431\u0443\u0434\u0435\u0442 \u043E\u0431\u0443\u0447\u0435\u043D \u043D\u0430 \u0431\u043E\u043B\u044C\u0448\u0435\u043C \u043D\u0430\u0431\u043E\u0440\u0435 \u0434\u0430\u043D\u043D\u044B\u0445, \u043D\u0435\u0436\u0435\u043B\u0438 \u0438\u0437\u043D\u0430\u0447\u0430\u043B\u044C\u043D\u043E. \u041D\u0435\u0439\u0440\u043E\u043D\u043D\u044B\u0435 \u0441\u0435\u0442\u0438 \u041D\u0435\u0439\u0440\u043E\u043D\u043D\u044B\u0435 \u0441\u0435\u0442\u0438 \u0441 \u043F\u0440\u044F\u043C\u043E\u0439 \u0441\u0432\u044F\u0437\u044C\u044E \u043F\u043E\u0437\u0432\u043E\u043B\u044F\u044E\u0442 \u0430\u043F\u043F\u0440\u043E\u043A\u0441\u0438\u043C\u0438\u0440\u043E\u0432\u0430\u0442\u044C \u0444\u0443\u043D\u043A\u0446\u0438\u0438, \u0447\u0442\u043E \u043F\u043E\u0437\u0432\u043E\u043B\u044F\u0435\u0442 \u0438\u0445 \u0438\u0441\u043F\u043E\u043B\u044C\u0437\u043E\u0432\u0430\u0442\u044C \u0432 \u0440\u0435\u0448\u0435\u043D\u0438\u0438 \u0437\u0430\u0434\u0430\u0447 \u043A\u043B\u0430\u0441\u0441\u0438\u0444\u0438\u043A\u0430\u0446\u0438\u0438. \u0422\u043E\u043F\u043E\u043B\u043E\u0433\u0438\u044F \u0442\u0430\u043A\u043E\u0439 \u0441\u0435\u0442\u0438 \u0445\u0430\u0440\u0430\u043A\u0442\u0435\u0440\u0438\u0437\u0443\u0435\u0442\u0441\u044F \u0442\u0435\u043C, \u0447\u0442\u043E \u043A\u043E\u043B\u0438\u0447\u0435\u0441\u0442\u0432\u043E \u043D\u0435\u0439\u0440\u043E\u043D\u043E\u0432 \u0432 \u0432\u044B\u0445\u043E\u0434\u043D\u043E\u043C \u0441\u043B\u043E\u0435, \u043A\u0430\u043A \u043F\u0440\u0430\u0432\u0438\u043B\u043E, \u0440\u0430\u0432\u043D\u043E \u043A\u043E\u043B\u0438\u0447\u0435\u0441\u0442\u0432\u0443 \u043E\u043F\u0440\u0435\u0434\u0435\u043B\u044F\u0435\u043C\u044B\u0445 \u043A\u0430\u0442\u0435\u0433\u043E\u0440\u0438\u0439 \u0421 \u0421 \u0421 &amp;(dj, \u0441,) = \u0422, \u0438 \u043D\u0435\u0433\u0430\u0442\u0438\u0432\u043D\u044B\u043C, \u0435\u0441\u043B\u0438 \u0424(dj, \u0441,) = F . \u0412 \u0443\u0441\u043B\u043E\u0432\u0438\u044F\u0445 \u0438\u0441\u0441\u043B\u0435\u0434\u043E\u0432\u0430\u043D\u0438\u044F \u0434\u043B\u044F \u043F\u043E\u0441\u0442\u0440\u043E\u0435\u043D\u043D\u043E\u0433\u043E \u043A\u043B\u0430\u0441\u0441\u0438\u0444\u0438\u043A\u0430\u0442\u043E\u0440\u0430 \u0424 \u0436\u0435\u043B\u0430\u0442\u0435\u043B\u044C\u043D\u043E \u0432\u044B\u0447\u0438\u0441\u043B\u0438\u0442\u044C \u0435\u0433\u043E \u044D\u0444\u0444\u0435\u043A\u0442\u0438\u0432\u043D\u043E\u0441\u0442\u044C. \u0412 \u0434\u0430\u043D\u043D\u043E\u043C \u0441\u043B\u0443\u0447\u0430\u0435, \u0434\u043E \u043F\u043E\u0441\u0442\u0440\u043E\u0435\u043D\u0438\u044F \u043A\u043B\u0430\u0441\u0441\u0438\u0444\u0438\u043A\u0430\u0442\u043E\u0440\u0430, \u0438\u0441\u0445\u043E\u0434\u043D\u044B\u0439 \u043D\u0430\u0431\u043E\u0440 \u0434\u043E\u043A\u0443\u043C\u0435\u043D\u0442\u043E\u0432 \u0440\u0430\u0437\u0431\u0438\u0432\u0430\u0435\u0442\u0441\u044F \u043D\u0430 \u0434\u0432\u0430 \u043D\u0430\u0431\u043E\u0440\u0430: \u2022 \u0442\u0440\u0435\u043D\u0438\u0440\u043E\u0432\u043E\u0447\u043D\u044B\u0439 \u043D\u0430\u0431\u043E\u0440 TV = [dlt... , d|TV|} . \u041D\u0430 \u043E\u0441\u043D\u043E\u0432\u0435 \u044D\u0442\u043E\u0433\u043E \u043D\u0430\u0431\u043E\u0440\u0430 \u0441\u0442\u0440\u043E\u0438\u0442\u0441\u044F \u043A\u043B\u0430\u0441\u0441\u0438\u0444\u0438\u043A\u0430\u0442\u043E\u0440 \u0424; \u2022 \u0442\u0435\u0441\u0442\u043E\u0432\u044B\u0439 \u043D\u0430\u0431\u043E\u0440 Te = (d|rF|&#43;\u0440..., d|Q |}, \u0438\u0441\u043F\u043E\u043B\u044C\u0437\u0443\u0435\u043C\u044B\u0439 \u0434\u043B\u044F \u0442\u0435\u0441\u0442\u0438\u0440\u043E\u0432\u0430\u043D\u0438\u044F \u044D\u0444\u0444\u0435\u043A\u0442\u0438\u0432\u043D\u043E\u0441\u0442\u0438 \u043A\u043B\u0430\u0441\u0441\u0438\u0444\u0438\u043A\u0430\u0442\u043E\u0440\u0430. \u0414\u043B\u044F \u043A\u0430\u0436\u0434\u043E\u0433\u043E \u0434\u043E\u043A\u0443\u043C\u0435\u043D\u0442\u0430 d. \xA3 Te , \u043E\u0431\u0440\u0430\u0431\u043E\u0442\u0430\u043D\u043D\u043E\u0433\u043E \u043A\u043B\u0430\u0441\u0441\u0438\u0444\u0438\u043A\u0430\u0442\u043E\u0440\u043E\u043C, \u0440\u0435\u0448\u0435\u043D\u0438\u0435 \u043A\u043B\u0430\u0441\u0441\u0438\u0444\u0438\u043A\u0430\u0442\u043E\u0440\u0430 \u0424(dj&#39;\u0441,) \u0441\u0440\u0430\u0432\u043D\u0438\u0432\u0430\u0435\u0442\u0441\u044F \u0441 \u044D\u043A\u0441\u043F\u0435\u0440\u0442\u043D\u044B\u043C \u0440\u0435\u0448\u0435\u043D\u0438\u0435\u043C , \u0441,) . \u041A\u043E\u043B\u0438\u0447\u0435\u0441\u0442\u0432\u0435\u043D\u043D\u043E\u0435 \u0438\u0437\u043C\u0435\u0440\u0435- \u043D\u0438\u0435 \u044D\u0444\u0444\u0435\u043A\u0442\u0438\u0432\u043D\u043E\u0441\u0442\u0438 \u043E\u0441\u043D\u043E\u0432\u0430\u043D\u043E \u043D\u0430 \u0432\u0435\u043B\u0438\u0447\u0438\u043D\u0435 \u0441\u043E\u0432\u043F\u0430\u0434\u0435\u043D\u0438\u0439 \u0424^.,, Cf) \u0438 \u0424(dj, Ct) . \u0414\u043E\u043A\u0443\u043C\u0435\u043D\u0442\u044B \u0438\u0437 \u043D\u0430\u0431\u043E\u0440\u0430 Te \u043D\u0435 \u0443\u0447\u0430\u0441\u0442\u0432\u0443\u044E\u0442 \u0432 \u0438\u043D\u0434\u0443\u043A\u0442\u0438\u0432\u043D\u043E\u043C \u043F\u0440\u043E\u0446\u0435\u0441\u0441\u0435 \u043F\u043E\u0441\u0442\u0440\u043E\u0435\u043D\u0438\u044F \u043A\u043B\u0430\u0441\u0441\u0438\u0444\u0438\u043A\u0430\u0442\u043E\u0440\u0430; nd P(dj/Cl) = P(tl, t2,..,/c1) = P(tx/Cl) \u2022 P(t2/c1 )...P(tnd |C ) =Y[P(tk/Cl) \u0423/\u0421/&gt; , \u0447\u0442\u043E \u0443\u0441\u0442\u0430\u043D\u0430\u0432\u043B\u0438\u0432\u0430\u0435\u0442 \u0441\u043E\u043E\u0442\u0432\u0435\u0442\u0441\u0442\u0432\u0438\u0435 \u043C\u0435\u0436\u0434\u0443 \u0432\u044B\u0445\u043E\u0434\u043E\u043C \u043D\u0435\u0439\u0440\u043E\u043D\u043D\u043E\u0439 \u0441\u0435\u0442\u0438 \u0438 \u043A\u0430\u0442\u0435\u0433\u043E\u0440\u0438\u0435\u0439 \u0421, \u043A\u043E\u0442\u043E\u0440\u0443\u044E \u043E\u043D \u043F\u0440\u0435\u0434\u0441\u0442\u0430\u0432\u043B\u044F\u0435\u0442. \u0422\u0430\u043A\u0438\u043C \u043E\u0431\u0440\u0430\u0437\u043E\u043C, \u0432 \u0441\u0435\u0442\u0438 \u0441 \u043E\u0434\u043D\u0438\u043C \u0441\u043A\u0440\u044B\u0442\u044B\u043C \u0441\u043B\u043E\u0435\u043C \u0432\u0435\u043A\u0442\u043E\u0440, \u0441\u043E\u043E\u0442\u0432\u0435\u0442\u0441\u0442\u0432\u0443\u044E\u0449\u0438\u0439 \u0432\u0445\u043E\u0434\u043D\u043E\u043C\u0443 \u043E\u0431\u0440\u0430\u0437\u0446\u0443, \u043F\u0440\u0435\u043E\u0431\u0440\u0430\u0437\u0443\u0435\u0442\u0441\u044F \u0441\u043A\u0440\u044B\u0442\u044B\u043C \u0441\u043B\u043E\u0435\u043C \u0432 \u043D\u0435\u043A\u043E\u0442\u043E\u0440\u043E\u0435 \u043D\u043E\u0432\u043E\u0435 \u043F\u0440\u043E\u0441\u0442\u0440\u0430\u043D\u0441\u0442\u0432\u043E, \u043A\u043E\u0442\u043E\u0440\u043E\u0435 \u043C\u043E\u0436\u0435\u0442 \u0438\u043C\u0435\u0442\u044C \u0434\u0440\u0443\u0433\u0443\u044E \u0440\u0430\u0437\u043C\u0435\u0440\u043D\u043E\u0441\u0442\u044C, \u0430 \u0437\u0430\u0442\u0435\u043C \u0433\u0438\u043F\u0435\u0440\u043F\u043B\u043E\u0441\u043A\u043E\u0441\u0442\u0438, \u0441\u043E\u043E\u0442\u0432\u0435\u0442\u0441\u0442\u0432\u0443\u044E\u0449\u0438\u0435 \u043D\u0435\u0439\u0440\u043E\u043D\u0430\u043C \u0432\u044B\u0445\u043E\u0434\u043D\u043E\u0433\u043E \u0441\u043B\u043E\u044F, \u0440\u0430\u0437\u0434\u0435\u043B\u044F\u044E\u0442 \u0435\u0433\u043E \u043D\u0430 \u043A\u043B\u0430\u0441\u0441\u044B [2]. \u041D\u0430\u0438\u0432\u043D\u0430\u044F \u0431\u0430\u0439\u0435\u0441\u043E\u0432\u0441\u043A\u0430\u044F \u043C\u043E\u0434\u0435\u043B\u044C \u0426\u0435\u043B\u044C \u043A\u043B\u0430\u0441\u0441\u0438\u0444\u0438\u043A\u0430\u0446\u0438\u0438 - \u043D\u0430\u0439\u0442\u0438 \u0441\u0430\u043C\u044B\u0439 \u043F\u043E\u0434\u0445\u043E\u0434\u044F\u0449\u0438\u0439 \u043A\u043B\u0430\u0441\u0441 \u0434\u043B\u044F \u0434\u0430\u043D\u043D\u043E\u0433\u043E \u0434\u043E\u043A\u0443\u043C\u0435\u043D\u0442\u0430. \u0412 \u043D\u0430\u0438\u0432\u043D\u043E\u0439 \u0431\u0430\u0439\u0435\u0441\u043E\u0432\u0441\u043A\u043E\u0439 \u043A\u043B\u0430\u0441\u0441\u0438\u0444\u0438\u043A\u0430\u0446\u0438\u0438 \u0437\u0430\u0434\u0430\u0447\u0430 \u0441\u043E\u0441\u0442\u043E\u0438\u0442 \u0432 \u043D\u0430\u0445\u043E\u0436\u0434\u0435\u043D\u0438\u0438 \u043D\u0430\u0438\u0431\u043E\u043B\u0435\u0435 \u0432\u0435\u0440\u043E\u044F\u0442\u043D\u043E\u0433\u043E \u043A\u043B\u0430\u0441\u0441\u0430 cm = argmax P(ci /dj ), \u0433\u0434\u0435 P( ci | dj ) - \u0432\u0435\u0440\u043E\u044F\u0442- ceC \u043D\u043E\u0441\u0442\u044C \u0442\u043E\u0433\u043E, \u0447\u0442\u043E \u0434\u043E\u043A\u0443\u043C\u0435\u043D\u0442 d. \u043F\u043E\u043F\u0430\u0434\u0451\u0442 \u0432 \u043A\u043B\u0430\u0441\u0441 \u0421 \u2022 \u0414\u043E\u043F\u0443\u0441\u043A\u0430\u044F \u0443\u043F\u0440\u043E\u0449\u0430\u044E\u0449\u0438\u0435 \u043F\u0440\u0435\u0434\u043F\u043E\u043B\u043E\u0436\u0435\u043D\u0438\u044F \u043E\u0431 \u0443\u0441\u043B\u043E\u0432\u043D\u043E\u0439 \u043D\u0435\u0437\u0430\u0432\u0438\u0441\u0438\u043C\u043E\u0441\u0442\u0438 \u0442\u0435\u0440\u043C\u043E\u0432 \u0438 \u043E \u043D\u0435\u0437\u0430\u0432\u0438\u0441\u0438\u043C\u043E\u0441\u0442\u0438 \u043F\u043E\u0437\u0438\u0446\u0438\u0439 \u0442\u0435\u0440\u043C\u043E\u0432 \u0438 \u0438\u0441\u043F\u043E\u043B\u044C\u0437\u0443\u044F \u043F\u0440\u0430\u0432\u0438\u043B\u043E \u0443\u043C\u043D\u043E\u0436\u0435\u043D\u0438\u044F \u0432\u0435\u0440\u043E\u044F\u0442\u043D\u043E\u0441\u0442\u0435\u0439 \u043D\u0435\u0437\u0430\u0432\u0438\u0441\u0438\u043C\u044B\u0445 \u0441\u043E\u0431\u044B\u0442\u0438\u0439, \u043C\u043E\u0436\u043D\u043E \u0437\u0430\u043F\u0438\u0441\u0430\u0442\u044C k=1 \u041E\u0446\u0435\u043D\u043A\u0430 \u0432\u0435\u0440\u043E\u044F\u0442\u043D\u043E\u0441\u0442\u0435\u0439 P(t / \u0441) \u0441 \u043F\u043E\u043C\u043E\u0449\u044C\u044E \u043E\u0431\u0443 \u0447\u0430\u044E\u0449\u0435\u0433\u043E \u043C\u043D\u043E\u0436\u0435\u0441\u0442\u0432\u0430 \u0431\u0443\u0434\u0435\u0442 P (t|c) = T-, \u0433\u0434\u0435 \u0422- - \u043A\u043E\u043B\u0438\u0447\u0435\u0441\u0442\u0432\u043E \u0432\u0445\u043E\u0436\u0434\u0435\u043D\u0438\u0439 \u0442\u0435\u0440\u043C\u0430 / \u0432\u043E \u0432\u0441\u0435\u0445 \u0434\u043E\u043A\u0443\u043C\u0435\u043D\u0442\u0430\u0445 \u043A\u043B\u0430\u0441\u0441\u0430 \u0441; \u0422\u0441 - \u043E\u0431\u0449\u0435\u0435 \u043A\u043E\u043B\u0438\u0447\u0435\u0441\u0442\u0432\u043E \u0442\u0435\u0440\u043C\u043E\u0432 \u0432 \u0434\u043E\u043A\u0443\u043C\u0435\u043D\u0442\u0430\u0445 \u043A\u043B\u0430\u0441\u0441\u0430 \u0441. \u041F\u043E\u0441\u043B\u0435 \u0442\u043E\u0433\u043E, \u043A\u0430\u043A \u043A\u043B\u0430\u0441\u0441\u0438\u0444\u0438\u043A\u0430\u0442\u043E\u0440 &#34;\u043E\u0431\u0443\u0447\u0435\u043D&#34;, \u0442\u043E \u0435\u0441\u0442\u044C, \u043D\u0430\u0439\u0434\u0435\u043D\u044B \u0432\u0435\u043B\u0438\u0447\u0438\u043D\u044B \u0420| \u0441) \u0438 \u0420(\u0441) , \u043C\u043E\u0436\u043D\u043E \u043E\u0442\u044B\u0441\u043A\u0430\u0442\u044C \u043A\u043B\u0430\u0441\u0441 \u0434\u043E\u043A\u0443\u043C\u0435\u043D\u0442\u0430 [1]: = argmax P (dj | ct )P (\u0441\u0433 ) = argmax P (ct )fjP(*k I \u0421 ) ceC ceC k=1 \u041A\u043B\u0430\u0441\u0442\u0435\u0440\u0438\u0437\u0430\u0446\u0438\u044F \u041A\u043B\u0430\u0441\u0442\u0435\u0440\u0438\u0437\u0430\u0446\u0438\u044F - \u0437\u0430\u0434\u0430\u0447\u0430 \u043C\u0430\u0448\u0438\u043D\u043D\u043E\u0433\u043E \u043E\u0431\u0443\u0447\u0435\u043D\u0438\u044F, \u0432 \u043A\u043E\u0442\u043E\u0440\u043E\u0439 \u0442\u0440\u0435\u0431\u0443\u0435\u0442\u0441\u044F \u0440\u0430\u0437\u0431\u0438\u0442\u044C \u0437\u0430\u0434\u0430\u043D\u043D\u0443\u044E \u0432\u044B\u0431\u043E\u0440\u043A\u0443 \u0434\u043E\u043A\u0443\u043C\u0435\u043D\u0442\u043E\u0432 \u043D\u0430 \u043D\u0435\u043F\u0435\u0440\u0435\u0441\u0435\u043A\u0430\u044E\u0449\u0438\u0435\u0441\u044F \u043F\u043E\u0434\u043C\u043D\u043E\u0436\u0435\u0441\u0442\u0432\u0430, \u043D\u0430\u0437\u044B- \u0432\u0430\u0435\u043C\u044B\u0435 \u043A\u043B\u0430\u0441\u0442\u0435\u0440\u0430\u043C\u0438, \u0442\u0430\u043A, \u0447\u0442\u043E\u0431\u044B \u043A\u0430\u0436\u0434\u044B\u0439 \u043A\u043B\u0430\u0441\u0442\u0435\u0440 \u0441\u043E\u0441\u0442\u043E\u044F\u043B \u0438\u0437 \u0441\u0445\u043E\u0436\u0438\u0445 \u043E\u0431\u044A\u0435\u043A\u0442\u043E\u0432, \u0430 \u043E\u0431\u044A\u0435\u043A\u0442\u044B \u0440\u0430\u0437\u043D\u044B\u0445 \u043A\u043B\u0430\u0441\u0442\u0435\u0440\u043E\u0432 \u0441\u0443\u0449\u0435\u0441\u0442\u0432\u0435\u043D\u043D\u043E \u043E\u0442\u043B\u0438\u0447\u0430\u043B\u0438\u0441\u044C. \u0420\u0435\u0448\u0435\u043D\u0438\u0435 \u0437\u0430\u0434\u0430\u0447\u0438 \u043A\u043B\u0430\u0441\u0442\u0435\u0440\u0438\u0437\u0430\u0446\u0438\u0438 \u043F\u0440\u0438\u043D\u0446\u0438\u043F\u0438\u0430\u043B\u044C\u043D\u043E \u043D\u0435\u043E\u0434\u043D\u043E\u0437\u043D\u0430\u0447\u043D\u043E, \u043F\u043E\u0441\u043A\u043E\u043B\u044C\u043A\u0443 \u043D\u0435 \u0441\u0443\u0449\u0435\u0441\u0442\u0432\u0443\u0435\u0442 \u043E\u0434\u043D\u043E\u0437\u043D\u0430\u0447\u043D\u043E \u043D\u0430\u0438\u043B\u0443\u0447\u0448\u0435\u0433\u043E \u0421 m \u043A\u0440\u0438\u0442\u0435\u0440\u0438\u044F \u043A\u0430\u0447\u0435\u0441\u0442\u0432\u0430 \u043A\u043B\u0430\u0441\u0442\u0435\u0440\u0438\u0437\u0430\u0446\u0438\u0438, \u0430 \u0442\u0430\u043A\u0436\u0435 \u0440\u0435\u0437\u0443\u043B\u044C\u0442\u0430\u0442 \u043A\u043B\u0430\u0441\u0442\u0435\u0440\u0438\u0437\u0430\u0446\u0438\u0438 \u0441\u0443\u0449\u0435\u0441\u0442\u0432\u0435\u043D\u043D\u043E \u0437\u0430\u0432\u0438\u0441\u0438\u0442 \u043E\u0442 \u043C\u0435\u0442\u0440\u0438\u043A\u0438 [3]. \u041D\u0430 \u043F\u0440\u0430\u043A\u0442\u0438\u043A\u0435 \u0447\u0430\u0441\u0442\u043E \u0438\u0441\u043F\u043E\u043B\u044C\u0437\u0443\u044E\u0442 \u043C\u043E\u0434\u0438\u0444\u0438\u043A\u0430\u0446\u0438\u044E \u0430\u043B\u0433\u043E\u0440\u0438\u0442\u043C\u0430 \u043A-\u0441\u0440\u0435\u0434\u043D\u0438\u0445 \u0438 \u0441-\u0441\u0440\u0435\u0434\u043D\u0438\u0445 [4]. \u0418\u043D\u0434\u0435\u043A\u0441\u0430\u0446\u0438\u044F \u0434\u043E\u043A\u0443\u043C\u0435\u043D\u0442\u043E\u0432 \u0414\u043E\u043A\u0443\u043C\u0435\u043D\u0442\u044B \u043D\u0435 \u043C\u043E\u0433\u0443\u0442 \u0431\u044B\u0442\u044C \u043D\u0430\u043F\u0440\u044F\u043C\u0443\u044E \u0438\u043D\u0442\u0435\u0440\u043F\u0440\u0435\u0442\u0438\u0440\u043E\u0432\u0430\u043D\u044B \u043A\u043B\u0430\u0441\u0441\u0438\u0444\u0438\u043A\u0430\u0442\u043E\u0440\u043E\u043C. \u041A\u0430\u043A \u043F\u0440\u0430\u0432\u0438\u043B\u043E, \u0434\u043E\u043A\u0443\u043C\u0435\u043D\u0442 \u0439 \u043F\u0440\u0435\u0434\u0441\u0442\u0430\u0432\u043B\u044F\u044E\u0442 \u0432 \u0432\u0438\u0434\u0435 \u0432\u0435\u043A\u0442\u043E\u0440\u0430 \u0432\u0435\u0441\u043E\u0432 \u0442\u0435\u0440\u043C\u043E\u0432 \u0433\u0434\u0435 T - \u043C\u043D\u043E\u0436\u0435\u0441\u0442\u0432\u043E \u0442\u0435\u0440\u043C\u043E\u0432, =(,-, \u2122/\u0422/&#39; \u0432\u0445\u043E\u0434\u044F\u0449\u0438\u0445, \u043F\u043E \u043C\u0435\u043D\u044C\u0448\u0435\u0439 \u043C\u0435\u0440\u0435, \u043E\u0434\u0438\u043D \u0440\u0430\u0437 \u0432 \u043E\u0434\u0438\u043D \u0434\u043E\u043A\u0443\u043C\u0435\u043D\u0442, \u0430 0 &lt; &lt; 1 \u043F\u0440\u0435\u0434\u0441\u0442\u0430\u0432\u043B\u044F\u0435\u0442 \u0441\u043E\u0431\u043E\u0439 \u043C\u0435\u0440\u0443 \u0442\u043E\u0433\u043E, \u043D\u0430 \u0441\u043A\u043E\u043B\u044C\u043A\u043E \u0442\u0435\u0440\u043C ^ \u043E\u043F\u0440\u0435\u0434\u0435\u043B\u044F\u0435\u0442 \u0441\u0435\u043C\u0430\u043D\u0442\u0438\u043A\u0443 \u0434\u043E\u043A\u0443\u043C\u0435\u043D\u0442\u0430 \u0439 . \u0412 \u0441\u043B\u0443\u0447\u0430\u0435 \u0438\u0441\u043F\u043E\u043B\u044C\u0437\u043E\u0432\u0430\u043D\u0438\u044F \u043D\u0435\u0434\u0432\u043E\u0438\u0447\u043D\u043E\u0439 \u0438\u043D\u0434\u0435\u043A\u0441\u0430\u0446\u0438\u0438, \u0434\u043B\u044F \u043E\u043F\u0440\u0435\u0434\u0435\u043B\u0435\u043D\u0438\u044F \u0432\u0435\u0441\u0430 wkj \u0442\u0435\u0440\u043C\u0430 ^ \u0434\u043E\u043A\u0443\u043C\u0435\u043D\u0442\u0430 \u0439 \u043C\u043E\u0436\u0435\u0442 \u0431\u044B\u0442\u044C \u0438\u0441\u043F\u043E\u043B\u044C\u0437\u043E\u0432\u0430\u043D \u043B\u044E\u0431\u043E\u0439 \u043C\u0435\u0442\u043E\u0434 \u0438\u0437\u0432\u043B\u0435\u0447\u0435\u043D\u0438\u044F \u0437\u043D\u0430\u043D\u0438\u0439, \u043E\u0441\u043D\u043E\u0432\u0430\u043D\u043D\u044B\u0439 \u043D\u0430 \u043F\u0440\u0435\u0434\u0441\u0442\u0430\u0432\u043B\u0435\u043D\u0438\u0438 \u0434\u043E\u043A\u0443\u043C\u0435\u043D\u0442\u0430 \u0432 \u0432\u0438\u0434\u0435 \u0432\u0435\u043A\u0442\u043E\u0440\u0430 \u0432\u0437\u0432\u0435\u0448\u0435\u043D\u043D\u044B\u0445 \u0442\u0435\u0440\u043C\u043E\u0432. \u0412 \u0431\u043E\u043B\u044C\u0448\u0438\u043D\u0441\u0442\u0432\u0435 \u0441\u043B\u0443\u0447\u0430\u0435\u0432 \u0438\u0441\u043F\u043E\u043B\u044C\u0437\u0443\u0435\u0442\u0441\u044F \u043C\u0435\u0442\u0440\u0438\u043A\u0430 tfidf. \u041E\u0434\u043D\u0430\u043A\u043E \u0432 \u043A\u043B\u0430\u0441\u0441\u0438\u0447\u0435\u0441\u043A\u043E\u0439 \u0435\u0435 \u0444\u043E\u0440\u043C\u0443\u043B\u0438\u0440\u043E\u0432\u043A\u0435, \u0434\u043E\u043A\u0443\u043C\u0435\u043D\u0442 \u043F\u0440\u0435\u0434\u0441\u0442\u0430\u0432\u043B\u044F\u0435\u0442\u0441\u044F \u043B\u0435\u043A\u0441\u0438\u0447\u0435\u0441\u043A\u043E\u0439 \u0441\u0435\u043C\u0430\u043D\u0442\u0438\u043A\u043E\u0439 \u0442\u0435\u0440\u043C\u043E\u0432, \u0435\u0433\u043E \u0441\u043E\u0441\u0442\u0430\u0432\u043B\u044F\u044E\u0449\u0438\u0445, \u0432 \u0442\u043E \u0432\u0440\u0435\u043C\u044F \u043A\u0430\u043A, \u0441\u043E\u0441\u0442\u0430\u0432\u043D\u0430\u044F \u0441\u0435\u043C\u0430\u043D\u0442\u0438\u043A\u0430 \u043D\u0435 \u0443\u0447\u0438\u0442\u044B\u0432\u0430\u0435\u0442\u0441\u044F. \u0414\u043B\u044F \u0440\u0435\u0448\u0435\u043D\u0438\u044F \u044D\u0442\u043E\u0439 \u043F\u0440\u043E\u0431\u043B\u0435\u043C\u044B \u0432\u0435\u0441\u0430, \u043F\u043E\u043B\u0443\u0447\u0435\u043D\u043D\u044B\u0435 \u043E\u0442 \u0444\u0443\u043D\u043A\u0446\u0438\u0438 tfidf, \u043D\u043E\u0440\u043C\u0430\u043B\u0438\u0437\u0443\u044E\u0442. \u0412 \u0437\u0430\u0432\u0438\u0441\u0438\u043C\u043E\u0441\u0442\u0438 \u043E\u0442 \u043F\u0440\u0430\u043A\u0442\u0438\u0447\u0435\u0441\u043A\u043E\u0433\u043E \u043F\u0440\u0438\u043C\u0435\u043D\u0435\u043D\u0438\u044F, \u043F\u0440\u043E\u0446\u0435\u0434\u0443\u0440\u0435 \u0438\u043D\u0434\u0435\u043A\u0441\u0430\u0446\u0438\u0438 \u043C\u043E\u0436\u0435\u0442 \u043F\u043E\u0434\u0432\u0435\u0440\u0433\u0430\u0442\u044C\u0441\u044F \u043F\u043E\u043B\u043D\u044B\u0439 \u0442\u0435\u043A\u0441\u0442 \u0434\u043E\u043A\u0443\u043C\u0435\u043D\u0442\u0430, \u043B\u0438\u0431\u043E \u0447\u0430\u0441\u0442\u0438 \u0434\u043E\u043A\u0443\u043C\u0435\u043D\u0442\u0430. \u0422\u0430\u043A\u0438\u043C \u043E\u0431\u0440\u0430\u0437\u043E\u043C, \u043F\u0440\u0438 \u043F\u043E\u0441\u0442\u0440\u043E\u0435\u043D\u0438\u0438 \u043A\u043B\u0430\u0441\u0441\u0438\u0444\u0438\u0446\u0438\u0440\u0443\u044E\u0449\u0438\u0445 \u0438\u043D\u0444\u043E\u0440\u043C\u0430\u0446\u0438\u043E\u043D\u043D\u044B\u0445 \u0441\u0438\u0441\u0442\u0435\u043C, \u043F\u0440\u0438\u043C\u0435\u043D\u0435\u043D\u0438\u0435 \u043A\u043B\u0430\u0441\u0441\u0438\u0447\u0435\u0441\u043A\u043E\u0433\u043E \u0430\u043B\u0433\u043E\u0440\u0438\u0442\u043C\u0430 \u043A\u043B\u0430\u0441\u0441\u0438\u0444\u0438\u043A\u0430\u0446\u0438\u0438 \u043C\u043E\u0436\u0435\u0442 \u0431\u044B\u0442\u044C \u043E\u043F\u0440\u0430\u0432\u0434\u0430\u043D\u043E \u0432 \u0441\u043B\u0443\u0447\u0430\u044F\u0445, \u043A\u043E\u0433\u0434\u0430 \u043E\u0431\u0443\u0447\u0430\u044E\u0449\u0438\u0435 \u0434\u0430\u043D\u043D\u044B\u0435 \u043D\u0430\u0438\u043B\u0443\u0447\u0448\u0438\u043C \u043E\u0431\u0440\u0430\u0437\u043E\u043C \u043E\u043F\u0438\u0441\u044B\u0432\u0430\u044E\u0442 \u0442\u0435 \u043A\u043B\u0430\u0441\u0441\u044B, \u043F\u043E \u043A\u043E\u0442\u043E\u0440\u044B\u043C \u043F\u0440\u043E\u0432\u043E\u0434\u0438\u043B\u043E\u0441\u044C \u043E\u0431\u0443\u0447\u0435\u043D\u0438\u0435. \u0420\u0435\u0448\u0435\u043D\u0438\u0435 \u043F\u0440\u043E\u0431\u043B\u0435\u043C\u044B \u043D\u0435\u043A\u0430\u0447\u0435\u0441\u0442\u0432\u0435\u043D\u043D\u044B\u0445 \u043E\u0431\u0443\u0447\u0430\u044E\u0449\u0438\u0445 \u0434\u0430\u043D\u043D\u044B\u0445 \u043C\u043E\u0436\u0435\u0442 \u0431\u044B\u0442\u044C \u0434\u043E\u0441\u0442\u0438\u0433\u043D\u0443\u0442\u043E, \u043F\u0440\u043E\u0432\u0435\u0434\u0435\u043D\u0438\u0435\u043C \u043F\u0440\u0435\u0434\u0432\u0430\u0440\u0438\u0442\u0435\u043B\u044C\u043D\u043E\u0433\u043E \u0440\u0430\u0437\u0431\u0438\u0435\u043D\u0438\u044F \u0434\u0430\u043D\u043D\u044B\u0445 \u0430\u043B\u0433\u043E\u0440\u0438\u0442\u043C\u043E\u043C \u043A\u043B\u0430\u0441\u0442\u0435\u0440\u0438\u0437\u0430\u0446\u0438\u0438. \u0412 \u0440\u0435\u0437\u0443\u043B\u044C\u0442\u0430\u0442\u0435 \u043A\u043B\u0430\u0441\u0442\u0435\u0440\u0438\u0437\u0430\u0446\u0438\u0438 \u043C\u043E\u0436\u0435\u0442 \u0431\u044B\u0442\u044C \u0441\u043D\u0438\u0436\u0435\u043D\u043E \u0432\u043B\u0438\u044F\u043D\u0438\u0435 \u0448\u0443\u043C\u0430 \u0432 \u043E\u0431\u0443\u0447\u0430\u044E\u0449\u0435\u0439 \u0432\u044B\u0431\u043E\u0440\u043A\u0435 \u043D\u0430 \u0440\u0435\u0437\u0443\u043B\u044C\u0442\u0430\u0442 \u043E\u0431\u0443\u0447\u0435\u043D\u0438\u044F \u0438, \u0432 \u043A\u043E\u043D\u0435\u0447\u043D\u043E\u043C \u0438\u0442\u043E\u0433\u0435, \u043F\u043E\u0441\u0442\u0440\u043E\u0435\u043D\u043D\u044B\u0439 \u043A\u043B\u0430\u0441\u0441\u0438\u0444\u0438\u043A\u0430\u0442\u043E\u0440. \u0421\u043F\u0438\u0441\u043E\u043A \u043B\u0438\u0442\u0435\u0440\u0430\u0442\u0443\u0440\u044B. 1. F. Sebastiani. Machine learning in automated text categorisation. ACM Computing Surveys, Vol. 34, No. 1., March 2002. 2. Christopher D. Manning, Prabhakar Raghavan, Hinrich Sch\xFCtze An Introduction to Information Retrieval Draft. Online edition. Cambridge University Press., 2009. 3. Stuart Russell, Peter Norvig, Artificial Intelligence: A Modern Approach, Prentice Hall, 2009. 4. David Arthur &amp; Sergei Vassilvitskii. &#34;How Slow is the k-means Method?&#34;. Proceedings of the 2006 Symposium on Computational Geometry (SoCG). -2006. \u0423\u0414\u041A 631.618 : 631.41: 631.48 &#43; 633.3: 631.5 \u0411\u043E\u0439\u043A\u043E \u0422.\u0413., \u041D\u0430\u0446\u0438\u043E\u043D\u0430\u043B\u044C\u043D\u044B\u0439 \u0443\u043D\u0438\u0432\u0435\u0440\u0441\u0438\u0442\u0435\u0442 \xAB\u041B\u044C\u0432\u043E\u0432\u0441\u043A\u0430\u044F \u043F\u043E\u043B\u0438\u0442\u0435\u0445\u043D\u0438\u043A\u0430\xBB, ORCID: https://orcid. org/0000-0002-048 7-3293 \u041F\u0430\u0441\u043B\u0430\u0432\u0441\u043A\u0438\u0439 \u041C.\u041C., \u041D\u0430\u0446\u0438\u043E\u043D\u0430\u043B\u044C\u043D\u044B\u0439 \u043B\u0435\u0441\u043E\u0442\u0435\u0445\u043D\u0438\u0447\u0435\u0441\u043A\u0438\u0439 \u0443\u043D\u0438\u0432\u0435\u0440\u0441\u0438\u0442\u0435\u0442 \u0423\u043A\u0440\u0430\u0438\u043D\u044B, ORCID: https://orcid. org/0000-0003-1635-4340 \u0420\u0443\u0434\u0430 \u041C.\u0412., \u041D\u0430\u0446\u0438\u043E\u043D\u0430\u043B\u044C\u043D\u044B\u0439 \u0443\u043D\u0438\u0432\u0435\u0440\u0441\u0438\u0442\u0435\u0442 \xAB\u041B\u044C\u0432\u043E\u0432\u0441\u043A\u0430\u044F \u043F\u043E\u043B\u0438\u0442\u0435\u0445\u043D\u0438\u043A\u0430\xBB, ORCID: https: /orcid.org/0000-0003-0590-4589 \u0420\u0420!: 10.24411/2520-6990-2019-10114 \u041C\u041E\u0414\u0415\u041B\u042C \u041E\u041F\u0420\u0415\u0414\u0415\u041B\u0415\u041D\u0418\u042F \u041D\u0410\u0414\u0415\u0416\u041D\u041E\u0421\u0422\u0418 \u041B\u0415\u0421\u041D\u041E\u0413\u041E \u041A\u041E\u041C\u041F\u0410\u0420\u0422\u041C\u0415\u041D\u0422\u0410 \u0421\u041B\u041E\u0416\u041D\u041E\u0413\u041E \u041B\u0410\u041D\u0414\u0428\u0410\u0424\u0422\u041D\u041E\u0413\u041E \u041A\u041E\u041C\u041F\u041B\u0415\u041A\u0421\u0410 \u0412 \u0414\u041D\u0415\u0421\u0422\u0420\u041E\u0412\u0421\u041A\u041E\u041C \u041F\u0420\u0418\u041A\u0410\u0420\u041F\u0410\u0422\u042C\u0415 Boyko T., National University &#34;Lviv Polytechnic&#34;, ORCID: https://orcid. org/0000-0002-048 7-3293 Paslavskyi M., Ukrainian National Forestry University, ORCID: https://orcid. org/0000-0003-1635-4340 Ruda M., National University &#34;Lviv Polytechnic&#34;, ORCID: https: /orcid.org/0000-0003-0590-4589 MODEL FOR DETERMINING THE RELIABILITY OF THE FOREST COMPARTMENT OF COMPOSITE LANDSCAPE COMPLEX OF THE DNIESTER&#39;S PRECARPATHIA \u0410\u043D\u043D\u043E\u0442\u0430\u0446\u0438\u044F. \u0412 \u0441\u0442\u0430\u0442\u044C\u0435 \u0440\u0430\u0441\u0441\u043C\u043E\u0442\u0440\u0435\u043D\u043E \u043F\u043E\u043D\u044F\u0442\u0438\u0435 \u043D\u0430\u0434\u0435\u0436\u043D\u043E\u0441\u0442\u0438 \u0431\u0438\u043E\u043B\u043E\u0433\u0438\u0447\u0435\u0441\u043A\u0438\u0445 \u0441\u0438\u0441\u0442\u0435\u043C, \u0432 \u0447\u0430\u0441\u0442\u043D\u043E\u0441\u0442\u0438 \u0441\u043B\u043E\u0436\u043D\u044B\u0445 \u043B\u0430\u043D\u0434\u0448\u0430\u0444\u0442\u043D\u044B\u0445 \u043A\u043E\u043C\u043F\u043B\u0435\u043A\u0441\u043E\u0432 (\u0421\u041B\u041A). \u0412 \u0421\u041B\u041A \u0432\u044B\u0434\u0435\u043B\u0435\u043D\u043E \u043A\u043E\u043C\u043F\u0430\u0440\u0442\u043C\u0435\u043D\u0442\u044B \u0438 \u043E\u043F\u0438\u0441\u0430\u043D\u043E \u0438\u0445 \u043F\u0440\u0438\u0440\u043E\u0434\u043D\u044B\u0435 \u043A\u043E\u043C\u043F\u043E\u043D\u0435\u043D\u0442\u044B \u0438 \u0430\u043D\u0442\u0440\u043E\u043F\u043E\u0433\u0435\u043D\u043D\u044B\u0435 \u0444\u0430\u043A\u0442\u043E\u0440\u044B. \u0412\u0430\u0436\u043D\u0435\u0439\u0448\u0435\u0439 \u0445\u0430\u0440\u0430\u043A\u0442\u0435\u0440\u0438\u0441\u0442\u0438\u043A\u043E\u0439 \u043F\u0440\u0438\u0440\u043E\u0434\u043E\u043E\u0445\u0440\u0430\u043D\u043D\u044B\u0445 \u043E\u0431\u044A\u0435\u043A\u0442\u043E\u0432, \u043F\u043E\u0437\u0432\u043E\u043B\u044F\u044E\u0449\u0435\u0439 \u043E\u043F\u0440\u0435\u0434\u0435\xBB\xBB' #@param {type: "string"}
article_path = '' #@param {type: "string"}

if (article_text == ''):
    with urllib.request.urlopen(article_path) as article_url:
      article_text = article_url.read().decode()

normalized_text = advanced_preprocess(article_text)
inferred_vector = model.infer_vector(normalized_text)
sims = model.docvecs.most_similar([inferred_vector], topn=5)

tabledata = [["Индекс", "Схожесть", "Текст"]]
print('\nТЕКСТ ИСХДНОГО ДОКУМЕНТА: «{}»\n'.format(article_text))
for label, index in [('1)', 0), ('2)', 1), ('3)', 2), ('4)', 3), ('5)', 4)]:
    tabledata.append([label, sims[index], get_article_text_by_id(int(sims[index][0]), train_path)])
print(tabulate(tabledata))